In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyvista as pv
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path

In [3]:
p  = !pwd
root=Path(p[0])

In [4]:
ROOT = root.parent
marv_obj = ROOT / "meshes/marv_lid.obj"
test_mesh = pv.read(marv_obj)

In [5]:
test_mesh.plot()

2024-12-09 11:54:36.405 Python[61096:16864505] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/yj/_6b0pqvs5xg33v_xy5q2v9_40000gn/T/org.python.python.savedState


Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x16a5da890_0&reconnect=auto" class="pyvista…

In [6]:
import yaml
with open(root.parent / f"robots/marv.yaml", "r") as file:
    robot_params = yaml.safe_load(file)

In [7]:
import torch
driving_part_bboxes = [torch.tensor(bbox) for bbox in robot_params["driving_part_bboxes"]]
body_bbox = torch.tensor(robot_params["body_bbox"])

In [8]:
robot_points = torch.tensor(pv.read(str(marv_obj)).points)

In [9]:
assert test_mesh.is_all_triangles

In [10]:
import pyacvd


def extract_flipper_points_from_mesh(mesh: pv.PolyData, n_points: int = 100, **clus_opts) -> torch.Tensor:
    delaunay = mesh.delaunay_3d()
    surf = delaunay.extract_surface()
    clus: pyacvd.Clustering = pyacvd.Clustering(surf)
    clus.cluster(n_points, **clus_opts)
    return torch.tensor(clus.cluster_centroid)

In [11]:
from flipper_training.utils.geometry import points_in_oriented_box
driving_meshes = []
driving_points = []
faces = test_mesh.faces.reshape(-1, 4)[:, 1:]
for p in driving_part_bboxes:
    mask = points_in_oriented_box(robot_points[:, :2], p)
    nonzero = mask.nonzero().numpy().flatten()
    pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)
    driving_meshes.append(pointgrid)
    driving_points.append(extract_flipper_points_from_mesh(pointgrid, 200))

2024-12-09 11:54:38.234 (   3.631s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:38.370 (   3.767s) [         10154F9]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x31c37ede0): 38 degenerate triangles encountered, mesh quality suspect
2024-12-09 11:54:38.593 (   3.991s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:38.724 (   4.122s) [         10154F9]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x31c408af0): 31 degenerate triangles encountered, mesh quality suspect
2024-12-09 11:54:38.960 (   4.358s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:39.100 (   4.497s) [         10154F9]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x333804080): 30 degenerate triangles encountered, mesh quality suspect
2024-12-09 11:54:39.335 (   4.732s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor

In [12]:
driving_meshes[0].plot(render_points_as_spheres=True)

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x31f54e9d0_1&reconnect=auto" class="pyvista…

In [13]:
delaunay = driving_meshes[0].delaunay_3d()
delaunay.plot()

2024-12-09 11:54:39.834 (   5.232s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:39.969 (   5.367s) [         10154F9]      vtkDelaunay3D.cxx:514   WARN| vtkDelaunay3D (0x10878d600): 38 degenerate triangles encountered, mesh quality suspect


Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x31f9f66d0_2&reconnect=auto" class="pyvista…

In [14]:
surface = delaunay.extract_surface()
surface_points = pv.PolyData(surface.points)
surface_points.plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x31f739290_3&reconnect=auto" class="pyvista…

In [15]:
import pyacvd

In [16]:
clus = pyacvd.Clustering(surface)
clus.cluster(200, maxiter=1000, iso_try=50)

array([  0,   1,   0, ..., 102,  42, 101], dtype=int32)

In [17]:
pv.PolyData(clus.cluster_centroid).plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x337e489d0_4&reconnect=auto" class="pyvista…

In [18]:
body_mask = points_in_oriented_box(robot_points[:, :2], body_bbox)
nonzero = body_mask.nonzero().numpy().flatten()
body_pointgrid = test_mesh.extract_points(nonzero, adjacent_cells=False, include_cells=True)

In [19]:
body_pointgrid.plot()

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x337ec08d0_5&reconnect=auto" class="pyvista…

In [20]:
body_delaunay = body_pointgrid.delaunay_3d()
body_delaunay.plot()

2024-12-09 11:54:41.289 (   6.686s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.686s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.686s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.686s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.686s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.687s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.687s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.687s) [         10154F9]            vtkMath.cxx:570   WARN| Unable to factor linear system
2024-12-09 11:54:41.289 (   6.68

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x16a5d8850_6&reconnect=auto" class="pyvista…

In [21]:
body_surface = body_pointgrid.extract_surface()
body_surface_points = pv.PolyData(body_surface.points)
body_surface_points.plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x349fddcd0_7&reconnect=auto" class="pyvista…

In [22]:
pv.PolyData(pv.voxelize(body_pointgrid.extract_surface(),density=0.1,check_surface=False).points).plot(render_points_as_spheres=True, point_size=10,)

Widget(value='<iframe src="http://localhost:51306/index.html?ui=P_0x35011be90_8&reconnect=auto" class="pyvista…